# ColdBoxParser demonstrator
ColdBoxParser is the interface for JSON strings provided from the Survey Monitoring System (SMS) of the ColdBox. 
It is written in C++, with the nlohmann::json library.

## Current status
Currently, it can only read monitoring and alarm messages from the SMS of the ColdBox.
As soon as I know what information needs in input, I will modify it to prepare JSON instructions towards the SMS.

In [1]:
.include /home/utente/DAQSNDBuild/software/external/json/single_include/nlohmann/

In [2]:
.L /home/utente/DAQSNDBuild/software/lib-host/monitoring/coldbox_parser.cpp

In [3]:
using json = nlohmann::json;

In [4]:
ColdBoxParser example

(ColdBoxParser &) @0x7f6d5d88b018


In [5]:
//update member string of parser, option false for 
example.UpdateStatusFromFile("/home/utente/cernbox/Synched/SNDOnline/DCS/monitoringexample.json", false);
//example.UpdateParametersFromFile("/home/utente/cernbox/Synched/SNDOnline/DCS/parameterexample.json"); 

In [6]:
example.GetTemperature (2) //from 1 to 5

(double) 20.306000


In [7]:
example.GetHumidity (2) //from 1 to 5

(double) 40.232000


In [8]:
example.IsSmokingTriggered(3) //from 1 to 3

(bool) false


In [9]:
example.IsCoolingOn()

(bool) true


In [10]:
example.GetAlertLevel()

(std::string) "monitoring"


In [11]:
example.GetStatus().dump(3)

(std::basic_string<char, std::char_traits<char>, std::allocator<char> >) "{
   "cooling": {
      "status": 1
   },
   "environment": {
      "t_rh-1": {
         "rh": 47.285,
         "t": 17.667
      },
      "t_rh-2": {
         "rh": 40.232,
         "t": 20.306
      },
      "t_rh-3": {
         "rh": 42.937,
         "t": 19.283
      },
      "t_rh-4": {
         "rh": 49.873,
         "t": 16.904
      },
      "t_rh-5": {
         "rh": 53.837,
         "t": 15.6
      }
   },
   "smoke": {
      "smk-1": {
         "triggered": 0
      },
      "smk-2": {
         "triggered": 0
      },
      "smk-3": {
         "triggered": 0
      }
   }
}"


## Output messages

How do we prepare a json to send messages? First, we need to know which parameters we can set:

In [12]:
example.PrintParList()

Printing list of parameters
0	alarm_min_n_t_rh
1	alarm_min_n_smk
2	alarm_t_thr_max
3	alarm_rh_thr_max
4	alarm_t_thr_min
5	alarm_rh_thr_min
6	monitor_update_time
7	alarm_update_time
8	cooling_grace_time
Use SetParameter(ipar,value) to set these parameters
Use ExcludeTemperatureSensor(isensor) and ExcludeSmokingSensor(isensor) to exclude a sensor


Then, we can set their values, for example setting maximum temperature and minimum number of sensors over threshold to trigger alarm. JSON does not differentiate between int and double

In [13]:

example.SetParameter(2,17.5)

In [14]:
example.SetParameter(0,1) 

we can also exclude some not working sensors

In [15]:
example.ExcludeTemperatureSensor(1)

In [16]:
example.ExcludeSmokingSensor(3)

In [17]:
example.SetParameter(2,18) //try to rechange the same parameter

In [18]:
cout<<example.GetSetParameters().dump(3)<<endl;

{
   "excluded_sensors": [
      "smk-3",
      "t_rh-1"
   ],
   "set_parameters": {
      "alarm_min_n_t_rh": 1.0,
      "alarm_t_thr_max": 18.0
   }
}


Let us assume we know the sensors now all work. How we reactivate them?

In [19]:
example.ActivateAllSensors();
cout<<example.GetSetParameters().dump(3)<<endl;

{
   "excluded_sensors": null,
   "set_parameters": {
      "alarm_min_n_t_rh": 1.0,
      "alarm_t_thr_max": 18.0
   }
}


### Clearing Parameters
Assume we want to clear our set of parameters, because we do not want to change them anymore. We can do

In [20]:
example.ClearParameters()

In [21]:
example.GetSetParameters().dump(3)

(std::basic_string<char, std::char_traits<char>, std::allocator<char> >) "{}"
